In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dropout, LeakyReLU, UpSampling2D
from tensorflow.keras.layers import Concatenate, Add
from tensorflow.keras.models import Model

In [2]:
# These values are arbitrarily chosen until network is finalised
batch_size = 64
depth = 16
categories = 2
epochs = 10
n = 128
m = 128

In [3]:
# Building the Model
def conv3x3(input_layer, filters, kernel_size=(3,3), strides=(1,1)):
    conv1 = Conv2D(filters, kernel_size, strides, padding="same", kernel_initializer="he_normal")(input_layer)
    leaky = LeakyReLU(0.01)(conv1)
    return leaky

def context_module(input_layer, filters):
    conv1 = conv3x3(input_layer, filters)
    drop1 = Dropout(0.3)(conv1)
    return conv3x3(drop1, filters)

def upsampling_module(input_layer, filters):
    upsample1 = UpSampling2D(size=(2,2))(input_layer)
    return conv3x3(upsample1, filters)

def localisation_module(input_layer, filters):
    conv1 = conv3x3(input_layer, filters)
    return conv3x3(conv1, filters, kernel_size=(1,1))

def build_model(input_shape, depth):
    inputs = Input(input_shape)
    
    # First Context Level
    conv1 = conv3x3(inputs, depth)
    context1 = context_module(conv1, depth)
    concat1 = Add()([conv1, context1])
    
    # Second Context Level
    conv2 = conv3x3(concat1, depth*2, strides=(2,2))
    context2 = context_module(conv2, depth*2)
    concat2 = Add()([conv2, context2])
    
    # Third Context Level
    conv3 = conv3x3(concat2, depth*4, strides=(2,2))
    context3 = context_module(conv3, depth*4)
    concat3 = Add()([conv3, context3])
    
    # Fourth Context Level
    conv4 = conv3x3(concat3, depth*8, strides=(2,2))
    context4 = context_module(conv4, depth*8)
    concat4 = Add()([conv4, context4])
    
    # Fifth (Last) Context Level
    conv5 = conv3x3(concat4, depth*16, strides=(2,2))
    context5 = context_module(conv5, depth*16)
    concat5 = Add()([conv5, context5])

    # Upsampling happens at this level too (bottom level)
    upsample1 = upsampling_module(concat5, depth*8)
    save1 = Concatenate()([upsample1, concat4])    
    localise1 = localisation_module(save1, depth*8)
    
    upsample2 = upsampling_module(localise1, depth*4)
    save2 = Concatenate()([upsample2, concat3])
    localise2 = localisation_module(save2, depth*4)
    
    seg1 = conv3x3(localise2, 16, kernel_size=(1,1))
    seg1 = UpSampling2D(size=(2,2))(seg1)
    
    upsample3 = upsampling_module(localise2, depth*2)
    save3 = Concatenate()([upsample3, concat2])
    localise3 = localisation_module(save3, depth*2)
    
    seg2 = conv3x3(localise3, 16, kernel_size=(1,1))
    seg2 = Add()([seg2, seg1])
    seg2 = UpSampling2D(size=(2,2))(seg2)
    
    upsample4 = upsampling_module(localise3, depth)
    save4 = Concatenate()([upsample4, concat1])
    
    conv_last = conv3x3(save4, depth*2)
    seg3 = conv3x3(conv_last, depth, kernel_size=(1,1))
    seg3 = Add()([seg3, seg2])
    
    softmax = Conv2D(categories, kernel_size=(1,1), padding="same", activation="softmax")(seg3)
    
    outputs = softmax
    model = Model(inputs, outputs)
    model.summary()
    
    return model 

In [4]:
model = build_model(input_shape=(n,m,categories), depth=depth)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 2) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 304         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 128, 128, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 2320        leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [5]:
def process_images(path, segmentation):
    return tf.keras.preprocessing.image_dataset_from_directory(
        directory = path, 
        labels=None,
        label_mode = 'binary',
        batch_size = batch_size,
        validation_split = 0.2,
        subset=segmentation,
        image_size = (n,m),
        color_mode = 'rgb',
        shuffle = True,
        seed = 45820188
    )

In [ ]:
# X_train_ds = process_images("C:\ISIC Dataset\Smaller\Train", "training")
# y_train_ds = process_images("C:\ISIC Dataset\Smaller\Seg", "training")

# X_test_ds = process_images("C:\ISIC Dataset\Smaller\Train", "validation")
# y_test_ds = process_images("C:\ISIC Dataset\Smaller\Seg", "validation")

X_train_ds = process_images("C:\ISIC Dataset\Full Set\ISIC2018_Task1-2_Training_Input_x2", "training")
y_train_ds = process_images("C:\ISIC Dataset\Full Set\ISIC2018_Task1_Training_GroundTruth_x2", "training")

X_test_ds = process_images("C:\ISIC Dataset\Full Set\ISIC2018_Task1-2_Training_Input_x2", "validation")
y_test_ds = process_images("C:\ISIC Dataset\Full Set\ISIC2018_Task1_Training_GroundTruth_x2", "validation")


X_train = None
for i in range(1, 3):
    batch = X_train_ds.take(i)
    for j in batch:
        X_train = j
        
y_train = None
for i in range(1, 3):
    batch = y_train_ds.take(i)
    for j in batch:
        y_train = j  
        
X_test = None
for i in range(1, 3):
    batch = X_test_ds.take(i)
    for j in batch:
        X_test = j
        
y_test = None
for i in range(1, 3)
    batch = y_test_ds.take(i)
    for j in batch:
        y_test = j
        
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# for images in X_train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.axis("off")
# plt.show()

# plt.figure(figsize=(10, 10))
# for images in y_train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.axis("off")
# plt.show()

In [ ]:
# import tensorflow_datasets as tfds

# X_train = None
# for image in tfds.as_numpy(X_train_ds):
#     X_train = image
    
# y_train = None
# for image in tfds.as_numpy(y_train_ds):
#     y_train = image
    
# X_test = None
# for image in tfds.as_numpy(X_test_ds):
#     X_test = image
    
# y_test = None
# for image in tfds.as_numpy(y_test_ds):
#     y_test = image

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
# print(type(X_train))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(X_test, y_test))